This code needs some debugging to be used:
- Statistically unbalanced: train set contains 46% positive data while 71% in test set
- need to save data as numpy array so that it can be read later (LOOK trial2)

In [49]:
import pycbc.noise
import pycbc.psd
import pycbc.filter
import pylab
from pycbc.filter import sigma
from pycbc.waveform import get_td_waveform
from pycbc.types.timeseries import load_timeseries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#import false signal functions
from ipynb.fs.full.falsig_functions import random_false_sig, flip_gw
from ipynb.fs.full.data_functions import noise, burried_gw, burried_false, burried_flip, visualize,bol, empty_gaussian

Trial 1 information:
- SNR 1000
- signals fully contained in windows of 1 second
- masses chosen to be integers for training data, with halves added on for test data, not randomly generated (specific masses shown below)
- no random seed fixed

Create a dataframe for pair of mass to iterate:

The following range guarentees that the whole waveform < around 0.8 so that it fits in the window.)
No partial signal considered yet

Use the following masses for training data;
For test data, subtract 1/2 from each mass. By using new set of mass, we can show model does not overfit and is able to make new predictions.

SCHEME:
upper bounds iterate: u = 75-100
- lower bound iterate l = 10-u

upper bounds iterate: u = 70-75
- lower bound iterate l = 12-u

upper bounds iterate: u = 65-70
- lower bound iterate l = 14-u

upper bounds iterate: u = 55-65
- lower bound iterate l = 16-u

upper bounds iterate: u = 50-55
- lower bound iterate l = 18-u

upper bounds iterate: u = 45-50
- lower bound iterate l = 20-u

upper bounds iterate: u = 40-45
- lower bound iterate l = 24-u

upper bounds iterate: u = 35-40
- lower bound iterate l = 28-u

upper bounds iterate: u = 30-35
- lower bound iterate l = 30-u

In [ ]:
# Training masses 
trainM = []
for u in np.arange(75,101):
    for l in np.arange(10,u+1):
        trainM.append([u,l])
for u in np.arange(70,75):
    for l in np.arange(12,u+1):
        trainM.append([u,l])
for u in np.arange(65,70):
    for l in np.arange(14,u+1):
        trainM.append([u,l])
for u in np.arange(55,65):
    for l in np.arange(16,u+1):
        trainM.append([u,l])
for u in np.arange(50,55):
    for l in np.arange(18,u+1):
        trainM.append([u,l])
for u in np.arange(45,50):
    for l in np.arange(20,u+1):
        trainM.append([u,l])
for u in np.arange(40,45):
    for l in np.arange(24,u+1):
        trainM.append([u,l])
for u in np.arange(35,40):
    for l in np.arange(28,u+1):
        trainM.append([u,l])
for u in np.arange(30,35):
    for l in np.arange(30,u+1):
        trainM.append([u,l]) 
trainM = pd.DataFrame(trainM)
trainM.columns = ['m1','m2']

# Testing masses
testM = trainM.copy()
testM = testM.apply(lambda x: x - 0.5)

# Use testM.sample(n) and trainM.sample(n) to randomly select a subset of n rows. 

In [51]:
# constants and variables
snr = 1000
mag = -21   # for false signals. Try to be close to the amplitude of gw signals
select1 = bol(4,len(trainM))
select2 = bol(4,1600)

In [53]:
# training_data
# Generate true cases that contain gw, classification label 1.
train = []

for i in tqdm(range(len(trainM))):
    wf = burried_gw(trainM.loc[i,'m1'], trainM.loc[i,'m2'], snr)
    train.append([wf,1])
    
    # Insert false flipped case as well as empty gaussian after every 4 true cases
    if select1[i]:
        train.append([burried_flip(trainM.loc[i,'m1'], trainM.loc[i,'m2'], snr),0])
        train.append([noise(wf,snr),0])


 14%|█▍        | 495/3536 [01:09<09:41,  5.23it/s]


 28%|██▊       | 995/3536 [02:18<07:30,  5.64it/s]


 35%|███▍      | 1226/3536 [02:51<06:17,  6.11it/s]


 49%|████▊     | 1722/3536 [1:09:18<6:58:53, 13.86s/it]


 62%|██████▏   | 2200/3536 [1:47:04<06:14,  3.57it/s]


 74%|███████▍  | 2622/3536 [1:48:07<01:57,  7.78it/s]


 86%|████████▋ | 3054/3536 [1:49:13<01:20,  6.00it/s]


 96%|█████████▋| 3411/3536 [1:50:16<00:24,  5.01it/s]


100%|██████████| 3536/3536 [1:50:36<00:00,  1.88s/it]

In [54]:
# Generate more data with empty gaussian on varying magitudes
for i in tqdm(range(20)):
    for l in np.arange(10):
        train.append([empty_gaussian(i/(10*l+1)),0])
    
# Generate false data with false signals on similar magnitude as gw signals
# insert signals on magnitude -20 and -22 after every 4 signals on magnitude -21
for i in tqdm(range(1600)):
    train.append([burried_false(mag,snr),0])
    
    if select2[i]:
        train.append([burried_false(-20,snr),0])
        train.append([burried_false(-22,snr),0])
                    


100%|██████████| 20/20 [00:08<00:00,  2.39it/s]

 18%|█▊        | 284/1600 [00:47<02:50,  7.71it/s]


 37%|███▋      | 586/1600 [01:38<02:53,  5.86it/s]


 56%|█████▌    | 888/1600 [05:01<02:06,  5.63it/s]


 74%|███████▍  | 1186/1600 [05:52<01:07,  6.15it/s]


 83%|████████▎ | 1335/1600 [06:15<00:50,  5.29it/s]


100%|██████████| 1600/1600 [07:04<00:00,  4.95it/s]


In [80]:
# Test data
testM_used = testM.sample(600)
testM_used = testM_used.reset_index()
test =[]

for i in tqdm(range(len(testM_used))):
    wf = burried_gw(testM_used.loc[i,'m1'], testM_used.loc[i,'m2'], snr)
    test.append([wf,1])
    
    # Insert false flipped case as well as empty gaussian after every 4 true cases
    if select1[i]:
        test.append([burried_flip(testM_used.loc[i,'m1'], testM_used.loc[i,'m2'], snr),0])
        test.append([noise(wf,snr),0])


 79%|███████▉  | 474/600 [01:10<00:15,  8.03it/s]


100%|██████████| 600/600 [01:27<00:00,  5.34it/s]


In [81]:
# Generate more data with empty gaussian on varying magitudes
for i in tqdm(range(10)):
    for l in np.arange(10):
        train.append([empty_gaussian((2*i+0.5)/(10*l+1.3)),0])
    
# Generate false data with false signals on similar magnitude as gw signals
# insert signals on magnitude -20 and -22 after every 4 signals on magnitude -21
for i in tqdm(range(240)):
    train.append([burried_false(mag,snr),0])
    
    if select2[i]:
        train.append([burried_false(-20,snr),0])
        train.append([burried_false(-22,snr),0])



100%|██████████| 10/10 [00:04<00:00,  2.34it/s]

100%|██████████| 240/240 [00:47<00:00,  4.66it/s]


In [87]:
# shuffle list
# needs to be randomly shuffled. Sequence of feeding matters
np.random.shuffle(train)
np.random.shuffle(test)

In [124]:
# Get trainX, trainY, testX, testY
train_df = pd.DataFrame(train, columns = ['ts' , 'target'])
trainX = train_df['ts']
trainY = train_df['target']

test_df = pd.DataFrame(test, columns = ['ts' , 'target'])
testX = test_df['ts']
testY = test_df['target']

In [126]:
# store 

''' ALREADY DONE. DON'T REPEAT
f_path = 'training_data/trial1/trainX.txt'
trainX.to_csv(f_path,sep=' ')
f_path = 'training_data/trial1/trainY.txt'
trainY.to_csv(f_path,sep=' ')
f_path = 'training_data/trial1/testX.txt'
testX.to_csv(f_path,sep=' ')
f_path = 'training_data/trial1/testY.txt'
testY.to_csv(f_path,sep=' ')
'''

when read from cvs, use following:

x = pd.read_csv(f_path, sep = ' ',names = ['drop','target'])

x = x.drop('drop',axis = 1)

In [153]:
# statistic information
print("Size of training data: ",len(train))
print("Size of test data: ",len(test))

train_pos = train_df[train_df['target'] >= 1]
print("Percentage ",100* len(train_pos)/len(train), "% of training data contains gw wave.")
test_pos = test_df[test_df['target'] >= 1]
print("Percentage ",100* len(test_pos)/len(test), "% of training data contains gw wave.")

Size of training data:  7826
Size of test data:  840
Percentage  45.182724252491695 % of training data contains gw wave.
Percentage  71.42857142857143 % of training data contains gw wave.
